In [1]:
import os

# Update with your API URL if using a hosted instance of Langsmith.
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com";
os.environ["LANGCHAIN_API_KEY"] = "" # Update with your API key
os.environ["OPENAI_API_KEY"] = ""

project_name = "default" # Update with your project name

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
("system", "You are world class technical documentation writer."),
("user", "{input}")
])

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

output = chain.invoke({"input": "how can langsmith help with testing?"})
print(output)

NameError: name 'llm' is not defined

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(os.environ["LANGCHAIN_ENDPOINT"])

In [6]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

loader = TextLoader("/Data/state_of_the_union.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

vector = FAISS.from_documents(documents, embeddings)
vector.save_local("/Data/faiss_index")
new_db = FAISS.load_local("/Data/faiss_index", embeddings, allow_dangerous_deserialization=True)

query = "How many barrels of oil is America contributing?"
docs = new_db.similarity_search_with_score(query)
print(docs)


[(Document(page_content='America will lead that effort, releasing 30 Million barrels from our own Strategic Petroleum Reserve. And we stand ready to do more if necessary, unified with our allies.  \n\nThese steps will help blunt gas prices here at home. And I know the news about what’s happening can seem alarming. \n\nBut I want you to know that we are going to be okay. \n\nWhen the history of this era is written Putin’s war on Ukraine will have left Russia weaker and the rest of the world stronger.', metadata={'source': '../Data/state_of_the_union.txt'}), 0.33403593), (Document(page_content='To all Americans, I will be honest with you, as I’ve always promised. A Russian dictator, invading a foreign country, has costs around the world. \n\nAnd I’m taking robust action to make sure the pain of our sanctions  is targeted at Russia’s economy. And I will use every tool at our disposal to protect American businesses and consumers. \n\nTonight, I can announce that the United States has worke